# Lab 5: LoRA Fine-tuning - SOLUTIONS

**Module 5 - Model Fine-tuning with Low-Rank Adaptation**

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
from datasets import Dataset
from trl import SFTTrainer

MODEL_NAME = "microsoft/phi-2"
print(f"CUDA available: {torch.cuda.is_available()}")

## Exercise 1: Dataset Preparation - SOLUTION

In [ ]:
def create_instruction_dataset():
    training_data = [
        {"instruction": "Explain machine learning simply.", "input": "", "output": "ML is AI that learns from data patterns."},
        {"instruction": "Summarize this text.", "input": "Transformers use attention.", "output": "Attention enables transformers."},
        {"instruction": "What is LoRA?", "input": "", "output": "LoRA trains small adapter matrices instead of full model weights."},
    ]
    return Dataset.from_list(training_data)

def format_instruction(example: dict, tokenizer) -> str:
    if example["input"]:
        return f"""### Instruction:\n{example["instruction"]}\n\n### Input:\n{example["input"]}\n\n### Response:\n{example["output"]}{tokenizer.eos_token}"""
    return f"""### Instruction:\n{example["instruction"]}\n\n### Response:\n{example["output"]}{tokenizer.eos_token}"""

dataset = create_instruction_dataset()
print(f"Dataset size: {len(dataset)}")

## Exercise 2: LoRA Configuration - SOLUTION

In [ ]:
def create_lora_config(r: int = 8, lora_alpha: int = 16) -> LoraConfig:
    return LoraConfig(
        r=r,
        lora_alpha=lora_alpha,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )

# Compare configurations
for r in [4, 8, 16, 32]:
    d, k = 2560, 2560  # Example dimensions
    original = d * k
    lora = r * (d + k)
    print(f"r={r}: LoRA params = {lora:,} vs Original = {original:,} ({original/lora:.1f}x reduction)")

## Exercise 3: Training Setup - SOLUTION

In [ ]:
def setup_training(model, tokenizer, dataset, output_dir="./lora_output"):
    lora_config = create_lora_config(r=8, lora_alpha=16)
    peft_model = get_peft_model(model, lora_config)
    
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=3,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        weight_decay=0.01,
        warmup_ratio=0.03,
        lr_scheduler_type="cosine",
        logging_steps=10,
        save_strategy="epoch",
        fp16=True,
        optim="paged_adamw_8bit",
        report_to="none",
    )
    
    # Format dataset
    def format_fn(examples):
        texts = [format_instruction({"instruction": examples["instruction"][i], "input": examples["input"][i], "output": examples["output"][i]}, tokenizer) for i in range(len(examples["instruction"]))]
        return {"text": texts}
    
    formatted = dataset.map(format_fn, batched=True, remove_columns=dataset.column_names)
    
    trainer = SFTTrainer(
        model=peft_model,
        train_dataset=formatted,
        args=training_args,
        tokenizer=tokenizer,
        dataset_text_field="text",
        max_seq_length=512,
    )
    return trainer, peft_model

print("Training setup function ready (requires GPU to run)")

## Exercise 4: Save and Load - SOLUTION

In [ ]:
def save_lora_adapter(model, output_path: str):
    model.save_pretrained(output_path)
    print(f"Adapter saved to {output_path}")

def load_lora_adapter(base_model_name: str, adapter_path: str):
    base_model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map="auto", trust_remote_code=True)
    model = PeftModel.from_pretrained(base_model, adapter_path)
    return model

def merge_and_save(model, tokenizer, output_path: str):
    merged = model.merge_and_unload()
    merged.save_pretrained(output_path)
    tokenizer.save_pretrained(output_path)
    print(f"Merged model saved to {output_path}")

print("Save/Load functions ready")

## Checkpoint

Lab 5 complete! **Next:** Lab 6 - Quantization & Optimization